**| /!\ LA METHODE 5 EST LA SEULE UTILE /!\ |**

# Library and DataFrame

In [1]:
import pandas as pd 
import numpy as np

In [2]:
df_april=pd.read_csv(r'azure_april_24.csv')
df_may=pd.read_csv(r'azure_may_24.csv')
df_vm=pd.read_csv(r'Informations_calcul - kWh_vm.csv')
df_geo=pd.read_csv(r'Informations_calcul - CO2_geo.csv')
df_ville=pd.read_csv(r'Informations_calcul - villes.csv')
df_price=pd.read_csv(r'Informations_calcul - price.csv')

C:\Users\saral\AppData\Local\Temp\ipykernel_13372\2607834369.py:1: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_april=pd.read_csv(r'azure_april_24.csv')
C:\Users\saral\AppData\Local\Temp\ipykernel_13372\2607834369.py:2: DtypeWarning: Columns (27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_may=pd.read_csv(r'azure_may_24.csv')


# Visualisation du DataFrame

In [3]:
df_may.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381693 entries, 0 to 381692
Data columns (total 55 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   InvoiceSectionName      381693 non-null  object 
 1   AccountName             381693 non-null  object 
 2   AccountOwnerId          381693 non-null  object 
 3   SubscriptionId          381693 non-null  object 
 4   SubscriptionName        381693 non-null  object 
 5   ResourceGroup           381159 non-null  object 
 6   ResourceLocation        381674 non-null  object 
 7   Date                    381693 non-null  object 
 8   ProductName             381693 non-null  object 
 9   MeterCategory           381693 non-null  object 
 10  MeterSubCategory        256294 non-null  object 
 11  MeterId                 381693 non-null  object 
 12  MeterName               381693 non-null  object 
 13  MeterRegion             186868 non-null  object 
 14  UnitOfMeasure       

Chaque DataFrame correspond au "ticket de caisse" de client.
Nous voulons nous, utiliser ces informations afin de calculer l'empreinte carbone de ce client. Pour cela nous choisissons les colonnes importantes et les renommons afin qu'elles collent avec celles utilisées par LotaCloud.

In [4]:
"""
Séléction des colonnes utilisées par lotacloud
"""
"""
Nous avons aussi ajouté 3 colonnes qui ous permetterons de calculer l'empreinte
carbonne : 
    CO2/kWh : Consomation de CO2 par kWh, cette information va être retrouvée
              grâce aux pays, en effet la colonne 'region' nous permet de 
              retrouver l'information.
    
    kWh : énergie utilisée par ressource.
          Cette information est plus difficile à trouver mais sans elle,
          il nous est impossible de calculer l'empreinte carbone.
          Pour calculer cette valeure nous avons utilisés plusieurs méthodes 
          que nous détaillerons par la suite
          
    CO2 : Gramme de CO2 relaché par ressource.
"""
def clean_columns(df) :
    df_copy = df.copy()
    df_copy = pd.DataFrame({ 
                     'usageAccountId' : df['AccountOwnerId'], 
                     'region' : df['ResourceLocation'], 
                     'region2' : df['MeterRegion'],
                     'endDate' : df['Date'],
                     'startDate' : df['Date'],
                     'productCode' : df['ConsumedService'],
                     'description' : df['ProductName'],
                     'usageType' : df['MeterName'],
                     'usageQuantityUnit' : df['UnitOfMeasure'],
                     'cost' : '',# : df_may['QuantityUnit'],
                     'usageQuantity' : df['Quantity'],
                     'costCenter' : df['CostCenter'],
                     'serviceCode' : df['MeterCategory'],
                     'tags' : df['Tags'],
                     'resourceId' : df['ResourceName'],
                     'provider' : df['PublisherType'],
                     'itemType' : df['ChargeType'],
                     'currency' : df['BillingCurrencyCode'],
                     'resourceGroup' : df['ResourceGroup'],
                     'subscriptionName' : df['SubscriptionName'],
                     'CO2/kWh' : 0,
                     'kWh' : 0,
                     'CO2' : 0,
                    })
    return df_copy



# Première méthode de calcul (pas utile)

Nous avons regroupé toutes les informations utiles à ce calcul dans un csv appelé 'Informations_calcul'. Ce csv contient 2 pages : **CO2_geo** : Tableau donnant l'emprunte carbone globale des data center par pays.
La deuxième page sera vu dans un second temps.

Pour la suite, nous avons travaillé sur 2 ressources différentes : les machines virtuelles et le stockage.

## Calcul kWh

Pour les machines virtuelles nous avons tout d'abord utiliser la moyenne, le max et le min de la puissance des machines virtuelles trouvée. Pour le stockage nous avons pu convertir l'unité GB en kWh pour nos calculs.

In [5]:
'''
Cette fonction va permettre d'ajouter 3 colonnes correspondant 
    à la moyenne de la puissance utilisé en kWh, le min et le max.

 Avec nos sources
 Nous avons choisis  :
     kW moyenne =  0.234
     kW max = 0.4
     kW min = 0.055
     0.005kW = 1GB
'''

"""
Ici nous ne remplissons pas la colonne kWh car nous avons choisit 3 valeurs 
différentes afin d'avoir une idée du comportement de la donnée
"""

def kWh_mmm(df) :
    df.loc[:,'Mean_kWh'] = 0
    df.loc[:,'Max_kWh'] = 0
    df.loc[:,'Min_kWh'] = 0
    
    #Pour VM
    df.loc[(df['usageQuantityUnit'] == "1 Hour") & 
           (df['serviceCode'].str.lower()=='virtual machines'), 'Mean_kWh'] = 0.234*df['usageQuantity']
    df.loc[(df['usageQuantityUnit'] == "1 Hour") & 
           (df['serviceCode'].str.lower()=='virtual machines'), 'Max_kWh'] = 0.4*df['usageQuantity']
    df.loc[(df['usageQuantityUnit'] == "1 Hour") & 
           (df['serviceCode'].str.lower()=='virtual machines'), 'Min_kWh'] = 0.055*df['usageQuantity']

    
    #Pour Storage
    df.loc[(df['usageQuantityUnit'] == "1 GB") & 
           (df['serviceCode'].str.lower()=='storage'), 'Mean_kWh'] = df['usageQuantity']*0.005
    
    df.loc[(df['usageQuantityUnit'] == "1 GB") & 
           (df['serviceCode'].str.lower()=='storage'), 'Max_kWh'] = df['usageQuantity']*0.005
    
    df.loc[(df['usageQuantityUnit'] == "1 GB") & 
           (df['serviceCode'].str.lower()=='storage'), 'Min_kWh'] = df['usageQuantity']*0.005
    
    
    df.loc[(df['usageQuantityUnit'] == "1 GB/Month") & 
           (df['serviceCode'].str.lower()=='storage'), 'Mean_kWh'] = df['usageQuantity']*0.005
    
    df.loc[(df['usageQuantityUnit'] == "1 GB/Month") & 
           (df['serviceCode'].str.lower()=='storage'), 'Max_kWh'] = df['usageQuantity']*0.005
    
    df.loc[(df['usageQuantityUnit'] == "1 GB/Month") & 
           (df['serviceCode'].str.lower()=='storage'), 'Min_kWh'] = df['usageQuantity']*0.005
    
    return df




## CO2/kWh

La fonction suivante ne dépend pas de la ressource mais de son emplacement géographique, en effet chaque pays produit de l'énergie différente et consomme une quantité différente de CO2, nous voulons donc associer chaque ressource au pays dans lequel son data center se situe.

In [6]:
"""
Ajout de l'information CO2/kWh dans sa colonne
"""
"""
Ici nous utilisons le dataframe contenant les informations du CO2 émit par kWh
"""

def co2_kWh(df, df_geo, month):
    df_copy = df.copy()
    df_geo_copy = df_geo.copy()
    for i in df_geo_copy['region'].value_counts().index :
        co2= df_geo_copy[f'{month} gCO2e/kwh'][df_geo_copy['region']==i]
        co2=co2.iloc[0]
        df_copy.loc[df_copy['region'] == i, 'CO2/kWh'] = co2
    
    return df_copy


## Calcul CO2

Enfin calculons son empreinte carbonne en multipliant les colonnes.

In [7]:
"""
Calcul pour le CO2 (mean, max, min)
"""
def co2_mmm(df):
    df_copy = df.copy()
    df_copy.loc[:,'Mean_CO2'] = 0
    df_copy.loc[:,'Max_CO2'] = 0
    df_copy.loc[:,'Min_CO2'] = 0
    df_copy.loc[:, 'Mean_CO2'] = df_copy['Mean_kWh']*df_copy['CO2/kWh']
    df_copy.loc[:, 'Max_CO2'] = df_copy['Max_kWh']*df_copy['CO2/kWh']
    df_copy.loc[:, 'Min_CO2'] = df_copy['Min_kWh']*df_copy['CO2/kWh']
    
    return df_copy

Utilisons maintenant toutes ces fonctions afin de créer notre premier csv contenant une approximation de l'emission de CO2.

In [8]:
def calcul_co2_mmm(df, df_geo, month) :
    #Copie des Data frame
        df_copy = df.copy()
        df_geo_copy = df_geo.copy()
        
    #Filtre de colonnes 
        df_copy = clean_columns(df_copy)
        
    #Calculs kWh
        df_copy = kWh_mmm(df_copy)
    
    #Ajout CO2/kWh
        df_copy = co2_kWh(df_copy, df_geo_copy, month)
        
    #Calcul CO2
        df_copy = co2_mmm(df_copy)
        
        return df_copy
    

## Création CSV pour graph

In [9]:
avril_df_mmm = calcul_co2_mmm(df_april, df_geo, 'april')
mai_df_mmm = calcul_co2_mmm(df_may, df_geo, 'may')

Afin de pouvoir visualiser nos données avec des graphiques nous voulons maintenant concatener ces deux dataframe, garder uniquement les lignes correspondant aux VM et au Storage. Pour éviter une trop grande table nous gardons uniquement quelques colonnes choisies auparavant

In [10]:
concat_avril_mai_mmm = pd.concat([avril_df_mmm, mai_df_mmm], ignore_index=True)
concat_avril_mai_mmm = concat_avril_mai_mmm[['region', 'endDate', 'startDate', 'productCode', 'usageType', 'usageQuantityUnit',
                              'usageQuantity', 'serviceCode', 'resourceId', 'itemType', 'CO2/kWh','Mean_kWh', 'Max_kWh',
                                'Min_kWh','Mean_CO2','Max_CO2','Min_CO2']]
concat_avril_mai_mmm= concat_avril_mai_mmm[(concat_avril_mai_mmm['serviceCode'].str.lower()=='virtual machines') | (concat_avril_mai_mmm['serviceCode'].str.lower()=='storage')]



On peut maintenant créer le csv et voir nos premiers graphiques

In [11]:
#concat_avril_mai_mmm.to_csv('méthode1.csv', index=False)

# Deuxième méthode de calcul (pas utile)

Dans cette partie nous utilisons le dataframe composé df_vm qui nous permet davoir les information sur l'energie de chaque machine virtuelle, nous n'utilisons donc pas la moyenne mais directement l'énergie utilisé par chaque machine virtuelle

## Fonctions utilisées précédement

In [12]:
"""
Séléction des colonnes utilisées par lotacloud
"""
def clean_columns(df) :
    df_copy = df.copy()
    df_copy = pd.DataFrame({ 
                     'usageAccountId' : df['AccountOwnerId'], 
                     'region' : df['ResourceLocation'], 
                     'region2' : df['MeterRegion'],
                     'endDate' : df['Date'],
                     'startDate' : df['Date'],
                     'productCode' : df['ConsumedService'],
                     'description' : df['ProductName'],
                     'usageType' : df['MeterName'],
                     'usageQuantityUnit' : df['UnitOfMeasure'],
                     'cost' : '',# : df_may['QuantityUnit'],
                     'usageQuantity' : df['Quantity'],
                     'costCenter' : df['CostCenter'],
                     'serviceCode' : df['MeterCategory'],
                     'tags' : df['Tags'],
                     'resourceId' : df['ResourceName'],
                     'provider' : df['PublisherType'],
                     'itemType' : df['ChargeType'],
                     'currency' : df['BillingCurrencyCode'],
                     'resourceGroup' : df['ResourceGroup'],
                     'subscriptionName' : df['SubscriptionName'],
                     'CO2/kWh' : 0,
                     'kWh' : 0,
                     'CO2' : 0,
                    })
    return df_copy

In [13]:
"""
Ajout de l'information CO2/kWh dans sa colonne
"""
def co2_kWh(df, df_geo, month):
    df_copy = df.copy()
    df_geo_copy = df_geo.copy()
    for i in df_geo_copy['region'].value_counts().index : # Verifier pour les valeures nul si pris en compte
        co2= df_geo_copy[f'{month} gCO2e/kwh'][df_geo_copy['region']==i]
        co2=co2.iloc[0]#Verifier si nombre
        df_copy.loc[df_copy['region'] == i, 'CO2/kWh'] = co2
    
    return df_copy

## Calcul du kWh pour les VM et le Storage

In [14]:
"""
Calcul du kWh pour les machines viruelles
"""
"""
Ici nous récupérons juste les valeure calculer dans le csv (RAM + CPU)
"""

def kWh_vm (df_client_1, df_vm_1) :
    df_client = df_client_1.copy()
    df_vm = df_vm_1.copy()
    for i in df_client['usageType'][df_client['serviceCode'].str.lower()=='virtual machines'].unique() :   
        #Filtrer sur le nom de la machine viruelle :
        if df_vm[df_vm['Name'] == i].size == 0 :
            break
        else :
            df_vm_filtred=df_vm[df_vm['Name'] == i]
            tdp=df_vm_filtred['TDP calculé']
            tdp=tdp.iloc[0]
            tdp = float(tdp.replace(',', '.'))### /!\ Tester si ca fonctionne car est ce que nombre ?
        #Ajout de l'info dans notre DF sur : le nom de la vm et sur les lignes vm
            df_client.loc[(df_client['usageQuantityUnit'] == "1 Hour") & 
                          (df_client['usageType'] == i) & 
                          (df_client['serviceCode'].str.lower()=='virtual machines')
                          , 'kWh']= tdp*df_client['usageQuantity']
    
    return df_client

In [15]:
"""
Calcul du kWh pour le storage
"""
def kWh_sto (df) :
    df_copy = df.copy()
    df_copy.loc[(df_copy['usageQuantityUnit'] == "1 GB") & 
    (df_copy['serviceCode'].str.lower()=='storage'), 'kWh'] = 0.005*df_copy['usageQuantity']
    df_copy.loc[(df_copy['usageQuantityUnit'] == "1 GB/Month") & 
    (df_copy['serviceCode'].str.lower()=='storage'), 'kWh'] = 0.005*df_copy['usageQuantity']
    
    return df_copy

## Calcul CO2

In [16]:
"""
Calcul pour le CO2
"""
def co2(df):
    df_copy = df.copy()
    df_copy.loc[:, 'CO2'] = df_copy['kWh']*df_copy['CO2/kWh']
    
    return df_copy

In [17]:
def calculation_co2_m2(df, df_geo, df_vm, month) :
    #Copie des Data frame
        df_copy = df.copy()
        df_geo_copy = df_geo.copy()
        df_vm_copy = df_vm.copy()
        
    #Filtre de colonnes 
        df_copy = clean_columns(df_copy)
        
    #Calculs kWh
        df_copy = kWh_vm(df_copy, df_vm_copy)
        df_copy = kWh_sto(df_copy)
    
    #Ajout CO2/kWh
        df_copy = co2_kWh(df_copy, df_geo_copy, month)
        
    #Calcul CO2
        df_copy = co2(df_copy)
        
        return df_copy

In [18]:
avril_df = calculation_co2_m2(df_april, df_geo, df_vm, 'april')
mai_df = calculation_co2_m2(df_may, df_geo, df_vm, 'may')

Concatenons une nouvelle fois nos dataframe et diminuons une nouvelle fois le nombre de ligne et colonne pour nos graphiques

In [19]:
concat_avril_mai = pd.concat([avril_df, mai_df], ignore_index=True)
concat_avril_mai = concat_avril_mai[['region', 'endDate', 'startDate', 'productCode', 'usageType', 'usageQuantityUnit',
                              'usageQuantity', 'serviceCode', 'resourceId', 'itemType', 'CO2/kWh','kWh','CO2']]
concat_avril_mai= concat_avril_mai[(concat_avril_mai['serviceCode'].str.lower()=='virtual machines') | (concat_avril_mai['serviceCode'].str.lower()=='storage')]

In [20]:
concat_avril_mai
#.to_csv('méthode2.csv', index=False)

,region,endDate,startDate,productCode,usageType,usageQuantityUnit,usageQuantity,serviceCode,resourceId,itemType,CO2/kWh,kWh,CO2
3,westeurope,04/09/2024,04/09/2024,Microsoft.Compute,P4 LRS Disk,1/Month,0.033336,Storage,gsebtsintkanweeumdk51,Usage,224,0.000000,0.00000
5,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Hot LRS Data Stored,1 GB/Month,0.009045,Storage,dypeaidevzzzfrctsto00,Usage,21,0.000045,0.00095
7,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,List Operations,10K,0.000300,Storage,shaeaiintshafrctsto01,Usage,21,0.000000,0.00000
10,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Read Operations,10K,9.803500,Storage,obheaidevzzzfrctsto00,Usage,21,0.000000,0.00000
14,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,All Other Operations,10K,5.869600,Storage,corapmprducrfrctsto00,Usage,21,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
729857,westeurope,05/31/2024,05/31/2024,Microsoft.Kusto,Write Operations,10K,0.019200,Storage,gsebdpprddatglwwadx01,Usage,228,0.000000,0.00000
729859,francecentral,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,corapmdevapmfrctsto00,Usage,17,0.000000,0.00000
729860,westeurope,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,pcmdgtdevoshweeusto00,Usage,228,0.000000,0.00000
729867,westeurope,05/31/2024,05/31/2024,Microsoft.Compute,E2 v3/E2s v3,1 Hour,24.000000,Virtual Machines,pcmdgtintspcweeuvmw00,Usage,228,3.336000,760.60800


# Troisième méthode

Dans la méthode précédentre nous prenions en compte le fait que chaque Core étaient utilisés pleinement, ici, grâce à l'information sur le nombre de Core et le nombre de CPU utilisés

## Fonctions utilisées précédement

In [21]:
"""
Séléction des colonnes utilisées par lotacloud
"""
def clean_columns(df) :
    df_copy = df.copy()
    df_copy = pd.DataFrame({ 
                     'usageAccountId' : df['AccountOwnerId'], 
                     'region' : df['ResourceLocation'], 
                     'region2' : df['MeterRegion'],
                     'endDate' : df['Date'],
                     'startDate' : df['Date'],
                     'productCode' : df['ConsumedService'],
                     'description' : df['ProductName'],
                     'usageType' : df['MeterName'],
                     'usageQuantityUnit' : df['UnitOfMeasure'],
                     'cost' : '',# : df_may['QuantityUnit'],
                     'usageQuantity' : df['Quantity'],
                     'costCenter' : df['CostCenter'],
                     'serviceCode' : df['MeterCategory'],
                     'tags' : df['Tags'],
                     'resourceId' : df['ResourceName'],
                     'provider' : df['PublisherType'],
                     'itemType' : df['ChargeType'],
                     'currency' : df['BillingCurrencyCode'],
                     'resourceGroup' : df['ResourceGroup'],
                     'subscriptionName' : df['SubscriptionName'],
                     'CO2/kWh' : 0,
                     'kWh' : 0,
                     'CO2' : 0,
                    })
    return df_copy

In [22]:
"""
Ajout de l'information CO2/kWh dans sa colonne
"""
def co2_kWh(df, df_geo, month):
    df_copy = df.copy()
    df_geo_copy = df_geo.copy()
    for i in df_geo_copy['region'].value_counts().index : # Verifier pour les valeures nul si pris en compte
        co2= df_geo_copy[f'{month} gCO2e/kwh'][df_geo_copy['region']==i]
        co2=co2.iloc[0]#Verifier si nombre
        df_copy.loc[df_copy['region'] == i, 'CO2/kWh'] = co2
    
    return df_copy

In [23]:
"""
Calcul du kWh pour le storage
"""
def kWh_sto (df) :
    df_copy = df.copy()
    df_copy.loc[(df_copy['usageQuantityUnit'] == "1 GB") & 
    (df_copy['serviceCode'].str.lower()=='storage'), 'kWh'] = 0.005*df_copy['usageQuantity']
    df_copy.loc[(df_copy['usageQuantityUnit'] == "1 GB/Month") & 
    (df_copy['serviceCode'].str.lower()=='storage'), 'kWh'] = 0.005*df_copy['usageQuantity']
    
    return df_copy

## Fonction kWh pour les VM modifié

In [24]:
"""
Calcul du kWh pour les machines viruelles
"""
"""
Ici nous récupérons juste les valeure calculer dans le csv (RAM + TDP*CPU/Core)
"""

def kWh_vm_met_3(df_client_1, df_vm_1) :
    df_client = df_client_1.copy()
    df_vm = df_vm_1.copy()
    for i in df_client['usageType'][df_client['serviceCode'].str.lower()=='virtual machines'].unique() :   
        #Filtrer sur le nom de la machine viruelle :
        if df_vm[df_vm['Name'] == i].size == 0 :
            break
        else :
            df_vm_filtred=df_vm[df_vm['Name'] == i]
            tdp=df_vm_filtred['TDP rec_kWh']
            tdp=tdp.iloc[0]
            tdp = float(tdp.replace(',', '.'))### /!\ Tester si ca fonctionne car est ce que nombre ?
        #Ajout de l'info dans notre DF sur : le nom de la vm et sur les lignes vm
            df_client.loc[(df_client['usageQuantityUnit'] == "1 Hour") & 
                          (df_client['usageType'] == i) & 
                          (df_client['serviceCode'].str.lower()=='virtual machines')
                          , 'kWh']= tdp*df_client['usageQuantity']
    
    return df_client

## Calcul CO2

In [25]:
"""
Calcul pour le CO2
"""
def co2(df):
    df_copy = df.copy()
    df_copy.loc[:, 'CO2'] = df_copy['kWh']*df_copy['CO2/kWh']
    
    return df_copy

def calculation_co2_met_3(df, df_geo, df_vm, month) :
    #Copie des Data frame
        df_copy = df.copy()
        df_geo_copy = df_geo.copy()
        df_vm_copy = df_vm.copy()
        
    #Filtre de colonnes 
        df_copy = clean_columns(df_copy)
        
    #Calculs kWh
        df_copy = kWh_vm_met_3(df_copy, df_vm_copy)
        df_copy = kWh_sto(df_copy)
    
    #Ajout CO2/kWh
        df_copy = co2_kWh(df_copy, df_geo_copy, month)
        
    #Calcul CO2
        df_copy = co2(df_copy)
        
        return df_copy

In [26]:
avril_df_met_3 = calculation_co2_met_3(df_april, df_geo, df_vm, 'april')
mai_df_met_3 = calculation_co2_met_3(df_may, df_geo, df_vm, 'may')

In [27]:
concat_avril_mai_met_3 = pd.concat([avril_df_met_3, mai_df_met_3], ignore_index=True)
concat_avril_mai_met_3 = concat_avril_mai_met_3[['region', 'endDate', 'startDate', 'productCode', 'usageType', 'usageQuantityUnit',
                              'usageQuantity', 'serviceCode', 'resourceId', 'itemType', 'CO2/kWh','kWh','CO2']]
concat_avril_mai_met_3= concat_avril_mai_met_3[(concat_avril_mai_met_3['serviceCode'].str.lower()=='virtual machines') | (concat_avril_mai_met_3['serviceCode'].str.lower()=='storage')]

In [28]:
concat_avril_mai_met_3
# concat_avril_mai_met_3.to_csv('méthode3.csv', index=False)

,region,endDate,startDate,productCode,usageType,usageQuantityUnit,usageQuantity,serviceCode,resourceId,itemType,CO2/kWh,kWh,CO2
3,westeurope,04/09/2024,04/09/2024,Microsoft.Compute,P4 LRS Disk,1/Month,0.033336,Storage,gsebtsintkanweeumdk51,Usage,224,0.000000,0.00000
5,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Hot LRS Data Stored,1 GB/Month,0.009045,Storage,dypeaidevzzzfrctsto00,Usage,21,0.000045,0.00095
7,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,List Operations,10K,0.000300,Storage,shaeaiintshafrctsto01,Usage,21,0.000000,0.00000
10,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Read Operations,10K,9.803500,Storage,obheaidevzzzfrctsto00,Usage,21,0.000000,0.00000
14,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,All Other Operations,10K,5.869600,Storage,corapmprducrfrctsto00,Usage,21,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
729857,westeurope,05/31/2024,05/31/2024,Microsoft.Kusto,Write Operations,10K,0.019200,Storage,gsebdpprddatglwwadx01,Usage,228,0.000000,0.00000
729859,francecentral,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,corapmdevapmfrctsto00,Usage,17,0.000000,0.00000
729860,westeurope,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,pcmdgtdevoshweeusto00,Usage,228,0.000000,0.00000
729867,westeurope,05/31/2024,05/31/2024,Microsoft.Compute,E2 v3/E2s v3,1 Hour,24.000000,Virtual Machines,pcmdgtintspcweeuvmw00,Usage,228,0.432000,98.49600


# Quatrième méthode (basée sur la 2) (pas utile)

## Fonctions utilisées précédement

In [29]:
"""
Séléction des colonnes utilisées par lotacloud
"""
def clean_columns(df) :
    df_copy = df.copy()
    df_copy = pd.DataFrame({ 
                     'usageAccountId' : df['AccountOwnerId'], 
                     'region' : df['ResourceLocation'], 
                     'region2' : df['MeterRegion'],
                     'endDate' : df['Date'],
                     'startDate' : df['Date'],
                     'productCode' : df['ConsumedService'],
                     'description' : df['ProductName'],
                     'usageType' : df['MeterName'],
                     'usageQuantityUnit' : df['UnitOfMeasure'],
                     'cost' : '',# : df_may['QuantityUnit'],
                     'usageQuantity' : df['Quantity'],
                     'costCenter' : df['CostCenter'],
                     'serviceCode' : df['MeterCategory'],
                     'tags' : df['Tags'],
                     'resourceId' : df['ResourceName'],
                     'provider' : df['PublisherType'],
                     'itemType' : df['ChargeType'],
                     'currency' : df['BillingCurrencyCode'],
                     'resourceGroup' : df['ResourceGroup'],
                     'subscriptionName' : df['SubscriptionName'],
                     'CO2/kWh' : 0,
                     'kWh' : 0,
                     'CO2' : 0,
                    })
    return df_copy

In [30]:
"""
Calcul du kWh pour les machines viruelles
"""
"""
Ici nous récupérons juste les valeure calculer dans le csv (RAM + CPU)
"""

def kWh_vm (df_client_1, df_vm_1) :
    df_client = df_client_1.copy()
    df_vm = df_vm_1.copy()
    for i in df_client['usageType'][df_client['serviceCode'].str.lower()=='virtual machines'].unique() :   
        #Filtrer sur le nom de la machine viruelle :
        if df_vm[df_vm['Name'] == i].size == 0 :
            break
        else :
            df_vm_filtred=df_vm[df_vm['Name'] == i]
            tdp=df_vm_filtred['TDP calculé']
            tdp=tdp.iloc[0]
            tdp = float(tdp.replace(',', '.'))### /!\ Tester si ca fonctionne car est ce que nombre ?
        #Ajout de l'info dans notre DF sur : le nom de la vm et sur les lignes vm
            df_client.loc[(df_client['usageQuantityUnit'] == "1 Hour") & 
                          (df_client['usageType'] == i) & 
                          (df_client['serviceCode'].str.lower()=='virtual machines')
                          , 'kWh']= tdp*df_client['usageQuantity']
    
    return df_client

In [31]:
"""
Calcul du kWh pour le storage
"""
def kWh_sto (df) :
    df_copy = df.copy()
    df_copy.loc[(df_copy['usageQuantityUnit'] == "1 GB") & 
    (df_copy['serviceCode'].str.lower()=='storage'), 'kWh'] = 0.005*df_copy['usageQuantity']
    df_copy.loc[(df_copy['usageQuantityUnit'] == "1 GB/Month") & 
    (df_copy['serviceCode'].str.lower()=='storage'), 'kWh'] = 0.005*df_copy['usageQuantity']
    
    return df_copy

In [32]:
"""
Calcul pour le CO2
"""
def co2_test(df):
    df_copy = df.copy()
    df_copy.loc[:, 'CO2'] = df_copy['kWh']*df_copy['CO2/kWh']
    
    return df_copy

## Fonction CO2 par ville

In [33]:
"""
Ajout de l'information CO2/kWh dans sa colonne
"""
def co2_kWh_city(df, df_geo):
    df_copy = df.copy()
    df_geo_copy = df_geo.copy()
    for i in df_geo_copy['region'].value_counts().index :# Verifier pour les valeures nul si pris en compte
        co2= df_geo_copy['ville'][df_geo_copy['region']==i]
        co2=co2.iloc[0]#Verifier si nombre
        co2 = float(co2.replace(',', '.'))
        df_copy.loc[df_copy['region'] == i, 'CO2/kWh'] = co2
    
    return df_copy

## Calcul CO2

In [34]:
def calculation_co2_m4(df, df_geo, df_vm) :
    #Copie des Data frame
        df_copy = df.copy()
        df_geo_copy = df_geo.copy()
        df_vm_copy = df_vm.copy()
        
    #Filtre de colonnes 
        df_copy = clean_columns(df_copy)
        
    #Calculs kWh
        df_copy = kWh_vm(df_copy, df_vm_copy)
        df_copy = kWh_sto(df_copy)
    
    #Ajout CO2/kWh
        df_copy = co2_kWh_city(df_copy, df_geo_copy)
        
    #Calcul CO2
        df_copy = co2_test(df_copy)
        
        return df_copy

In [35]:
avril_df_m4 = calculation_co2_m4(df_april, df_geo, df_vm)
mai_df_m4 = calculation_co2_m4(df_may, df_geo, df_vm)

In [36]:
concat_avril_mai_m4 = pd.concat([avril_df_m4, mai_df_m4], ignore_index=True)
concat_avril_mai_m4 = concat_avril_mai_m4[['region', 'endDate', 'startDate', 'productCode', 'usageType', 'usageQuantityUnit',
                              'usageQuantity', 'serviceCode', 'resourceId', 'itemType', 'CO2/kWh','kWh','CO2']]
concat_avril_mai_m4= concat_avril_mai_m4[(concat_avril_mai_m4['serviceCode'].str.lower()=='virtual machines') | (concat_avril_mai_m4['serviceCode'].str.lower()=='storage')]

In [37]:
concat_avril_mai_m4
# concat_avril_mai.to_csv('méthode4.csv', index=False)

,region,endDate,startDate,productCode,usageType,usageQuantityUnit,usageQuantity,serviceCode,resourceId,itemType,CO2/kWh,kWh,CO2
3,westeurope,04/09/2024,04/09/2024,Microsoft.Compute,P4 LRS Disk,1/Month,0.033336,Storage,gsebtsintkanweeumdk51,Usage,328.40,0.000000,0.000000
5,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Hot LRS Data Stored,1 GB/Month,0.009045,Storage,dypeaidevzzzfrctsto00,Usage,51.28,0.000045,0.002319
7,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,List Operations,10K,0.000300,Storage,shaeaiintshafrctsto01,Usage,51.28,0.000000,0.000000
10,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Read Operations,10K,9.803500,Storage,obheaidevzzzfrctsto00,Usage,51.28,0.000000,0.000000
14,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,All Other Operations,10K,5.869600,Storage,corapmprducrfrctsto00,Usage,51.28,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
729857,westeurope,05/31/2024,05/31/2024,Microsoft.Kusto,Write Operations,10K,0.019200,Storage,gsebdpprddatglwwadx01,Usage,328.40,0.000000,0.000000
729859,francecentral,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,corapmdevapmfrctsto00,Usage,51.28,0.000000,0.000000
729860,westeurope,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,pcmdgtdevoshweeusto00,Usage,328.40,0.000000,0.000000
729867,westeurope,05/31/2024,05/31/2024,Microsoft.Compute,E2 v3/E2s v3,1 Hour,24.000000,Virtual Machines,pcmdgtintspcweeuvmw00,Usage,328.40,3.336000,1095.542400


# Cinquième méthode (basée sur la 3)

## Fonctions utilisées précédement

In [38]:
"""
Séléction des colonnes utilisées par lotacloud
"""
def clean_columns(df) :
    df_copy = df.copy()
    df_copy = pd.DataFrame({ 
                     'usageAccountId' : df['AccountOwnerId'], 
                     'region' : df['ResourceLocation'], 
                     'region2' : df['MeterRegion'],
                     'endDate' : df['Date'],
                     'startDate' : df['Date'],
                     'productCode' : df['ConsumedService'],
                     'description' : df['ProductName'],
                     'usageType' : df['MeterName'],
                     'usageQuantityUnit' : df['UnitOfMeasure'],
                     'cost' : '',# : df_may['QuantityUnit'],
                     'usageQuantity' : df['Quantity'],
                     'costCenter' : df['CostCenter'],
                     'serviceCode' : df['MeterCategory'],
                     'tags' : df['Tags'],
                     'resourceId' : df['ResourceName'],
                     'provider' : df['PublisherType'],
                     'itemType' : df['ChargeType'],
                     'currency' : df['BillingCurrencyCode'],
                     'resourceGroup' : df['ResourceGroup'],
                     'subscriptionName' : df['SubscriptionName'],
                     'CO2/kWh' : 0,
                     'kWh' : 0,
                     'CO2' : 0,
                    })
    return df_copy

In [39]:
"""
Calcul du kWh pour le storage
"""
def kWh_sto (df) :
    df_copy = df.copy()
    df_copy.loc[(df_copy['usageQuantityUnit'] == "1 GB") & 
    (df_copy['serviceCode'].str.lower()=='storage'), 'kWh'] = 0.005*df_copy['usageQuantity']
    df_copy.loc[(df_copy['usageQuantityUnit'] == "1 GB/Month") & 
    (df_copy['serviceCode'].str.lower()=='storage'), 'kWh'] = 0.005*df_copy['usageQuantity']
    
    return df_copy

In [40]:
"""
Calcul du kWh pour les machines viruelles
"""
"""
Ici nous récupérons juste les valeure calculer dans le csv (RAM + TDP*CPU/Core)
"""

def kWh_vm_met_3(df_client_1, df_vm_1) :
    df_client = df_client_1.copy()
    df_vm = df_vm_1.copy()
    for i in df_client['usageType'][df_client['serviceCode'].str.lower()=='virtual machines'].unique() :   
        #Filtrer sur le nom de la machine viruelle :
        if df_vm[df_vm['Name'] == i].size == 0 :
            break
        else :
            df_vm_filtred=df_vm[df_vm['Name'] == i]
            tdp=df_vm_filtred['TDP rec_kWh']
            tdp=tdp.iloc[0]
            tdp = float(tdp.replace(',', '.'))### /!\ Tester si ca fonctionne car est ce que nombre ?
        #Ajout de l'info dans notre DF sur : le nom de la vm et sur les lignes vm
            df_client.loc[(df_client['usageQuantityUnit'] == "1 Hour") & 
                          (df_client['usageType'] == i) & 
                          (df_client['serviceCode'].str.lower()=='virtual machines')
                          , 'kWh']= tdp*df_client['usageQuantity']
    
    return df_client

## Fonction CO2 par ville

In [41]:
"""
Ajout de l'information CO2/kWh dans sa colonne
"""
def co2_kWh_city(df, df_geo):
    df_copy = df.copy()
    df_geo_copy = df_geo.copy()
    for i in df_geo_copy['region'].value_counts().index :# Verifier pour les valeures nul si pris en compte
        co2= df_geo_copy['ville'][df_geo_copy['region']==i]
        co2=co2.iloc[0]#Verifier si nombre
        co2 = float(co2.replace(',', '.'))
        df_copy.loc[df_copy['region'] == i, 'CO2/kWh'] = co2
    
    return df_copy

## Calcul CO2

In [42]:
"""
Calcul pour le CO2
"""
def co2(df):
    df_copy = df.copy()
    df_copy.loc[:, 'CO2'] = df_copy['kWh']*df_copy['CO2/kWh']
    
    return df_copy

def calculation_co2_met_5(df, df_geo, df_vm) :
    #Copie des Data frame
        df_copy = df.copy()
        df_geo_copy = df_geo.copy()
        df_vm_copy = df_vm.copy()
        
    #Filtre de colonnes 
        df_copy = clean_columns(df_copy)
        
    #Calculs kWh
        df_copy = kWh_vm_met_3(df_copy, df_vm_copy)
        df_copy = kWh_sto(df_copy)
    
    #Ajout CO2/kWh
        df_copy = co2_kWh_city(df_copy, df_geo_copy)
        
    #Calcul CO2
        df_copy = co2_test(df_copy)
        
        return df_copy

In [43]:
avril_df_m5 = calculation_co2_met_5(df_april, df_geo, df_vm)
mai_df_m5 = calculation_co2_met_5(df_may, df_geo, df_vm)

In [44]:
concat_avril_mai_m5 = pd.concat([avril_df_m5, mai_df_m5], ignore_index=True)
concat_avril_mai_m5 = concat_avril_mai_m5[['region', 'endDate', 'startDate', 'productCode', 'usageType', 'usageQuantityUnit',
                              'usageQuantity', 'serviceCode', 'resourceId', 'itemType', 'CO2/kWh','kWh','CO2']]
concat_avril_mai_m5= concat_avril_mai_m5[(concat_avril_mai_m5['serviceCode'].str.lower()=='virtual machines') | (concat_avril_mai_m5['serviceCode'].str.lower()=='storage')]

In [45]:
concat_avril_mai_m5
# concat_avril_mai_m5.to_csv('méthode5.csv', index=False)

,region,endDate,startDate,productCode,usageType,usageQuantityUnit,usageQuantity,serviceCode,resourceId,itemType,CO2/kWh,kWh,CO2
3,westeurope,04/09/2024,04/09/2024,Microsoft.Compute,P4 LRS Disk,1/Month,0.033336,Storage,gsebtsintkanweeumdk51,Usage,328.40,0.000000,0.000000
5,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Hot LRS Data Stored,1 GB/Month,0.009045,Storage,dypeaidevzzzfrctsto00,Usage,51.28,0.000045,0.002319
7,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,List Operations,10K,0.000300,Storage,shaeaiintshafrctsto01,Usage,51.28,0.000000,0.000000
10,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Read Operations,10K,9.803500,Storage,obheaidevzzzfrctsto00,Usage,51.28,0.000000,0.000000
14,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,All Other Operations,10K,5.869600,Storage,corapmprducrfrctsto00,Usage,51.28,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
729857,westeurope,05/31/2024,05/31/2024,Microsoft.Kusto,Write Operations,10K,0.019200,Storage,gsebdpprddatglwwadx01,Usage,328.40,0.000000,0.000000
729859,francecentral,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,corapmdevapmfrctsto00,Usage,51.28,0.000000,0.000000
729860,westeurope,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,pcmdgtdevoshweeusto00,Usage,328.40,0.000000,0.000000
729867,westeurope,05/31/2024,05/31/2024,Microsoft.Compute,E2 v3/E2s v3,1 Hour,24.000000,Virtual Machines,pcmdgtintspcweeuvmw00,Usage,328.40,0.432000,141.868800


# M4 et M5 en une fonction / (pas utile)

Nous allons ici faire une fonction qui ajoute une colonne pour chaque méthode 

In [46]:
def m4_m5 (df, df_geo, df_vm) :
    df_final = df.copy()
    df_m4 = df.copy()
    df_m5 = df.copy()
    
    df_final=clean_columns(df_final)
    
    df_m4 = calculation_co2_m4(df_m4, df_geo, df_vm)
    df_m5 = calculation_co2_met_5(df_m5, df_geo, df_vm)
    
    df_final=df_final.drop('CO2/kWh', axis=1)
    df_final=df_final.drop('kWh', axis=1)
    df_final=df_final.drop('CO2', axis=1)
    
    df_final['CO2_methode_4']= df_m4['CO2']
    df_final['CO2_methode_5']= df_m5['CO2']
    
    return df_final
    
   

In [47]:
m4m5_april=m4_m5 (df_april, df_geo, df_vm)
m4m5_may=m4_m5 (df_may, df_geo, df_vm)

In [48]:
concat_m4m5 = pd.concat([m4m5_april, m4m5_may], ignore_index=True)
concat_m4m5 = concat_m4m5[['region', 'endDate', 'startDate', 'productCode', 'usageType', 'usageQuantityUnit',
                              'usageQuantity', 'serviceCode', 'resourceId', 'itemType', 'CO2_methode_4','CO2_methode_5']]
concat_m4m5= concat_m4m5[(concat_m4m5['serviceCode'].str.lower()=='virtual machines') | (concat_m4m5['serviceCode'].str.lower()=='storage')]

In [49]:
concat_m4m5
# concat_m4m5.to_csv('méthode4_5.csv', index=False)

,region,endDate,startDate,productCode,usageType,usageQuantityUnit,usageQuantity,serviceCode,resourceId,itemType,CO2_methode_4,CO2_methode_5
3,westeurope,04/09/2024,04/09/2024,Microsoft.Compute,P4 LRS Disk,1/Month,0.033336,Storage,gsebtsintkanweeumdk51,Usage,0.000000,0.000000
5,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Hot LRS Data Stored,1 GB/Month,0.009045,Storage,dypeaidevzzzfrctsto00,Usage,0.002319,0.002319
7,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,List Operations,10K,0.000300,Storage,shaeaiintshafrctsto01,Usage,0.000000,0.000000
10,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,Read Operations,10K,9.803500,Storage,obheaidevzzzfrctsto00,Usage,0.000000,0.000000
14,francecentral,04/09/2024,04/09/2024,Microsoft.Storage,All Other Operations,10K,5.869600,Storage,corapmprducrfrctsto00,Usage,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
729857,westeurope,05/31/2024,05/31/2024,Microsoft.Kusto,Write Operations,10K,0.019200,Storage,gsebdpprddatglwwadx01,Usage,0.000000,0.000000
729859,francecentral,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,corapmdevapmfrctsto00,Usage,0.000000,0.000000
729860,westeurope,05/31/2024,05/31/2024,Microsoft.Storage,LRS List and Create Container Operations,10K,0.013500,Storage,pcmdgtdevoshweeusto00,Usage,0.000000,0.000000
729867,westeurope,05/31/2024,05/31/2024,Microsoft.Compute,E2 v3/E2s v3,1 Hour,24.000000,Virtual Machines,pcmdgtintspcweeuvmw00,Usage,1095.542400,141.868800


# Fonction Prix 

Le but ici est de pouvoir mettre en avant le prix des data centers afin de pouvoir le comparer avec l'emission de CO2

In [50]:
"""
Attention ici il le dataframe entrant devra déja être passé par une des méthodes pour fonctionner
"""

def prix_vm (df, df_price) :
    df_copy = df.copy()
    df_price_copy = df_price.copy()
    df_copy.loc[:,'price']=0

    for i in df_copy['usageType'][df_copy['serviceCode'].str.lower()=='virtual machines'].unique() :   
        for j in df_copy['region'][(df_copy['serviceCode'].str.lower()=='virtual machines') & (df_copy['usageType']==i)].unique():

            result=df_price_copy[j][df_price_copy['Name'] == i] #filtrer sur la machine virtuelle
            result=result.iloc[0]
            result = float(result.replace(',', '.'))
            

            df_copy.loc[(df_copy['serviceCode'].str.lower()=='virtual machines') & 
                          (df_copy['usageType'] == i) & 
                          (df_copy['region'] == j)
                          , 'price']= result*df_copy['usageQuantity']
    
    return df_copy
    
    
    

In [51]:
prix_m4_m5=prix_vm(concat_m4m5, df_price)

In [52]:
# prix_m4_m5.to_csv('prix_méthode4_5.csv', index=False)

# Fonction CO2 par région

On veut ici avoir la possibilité d'ajouter une colone qui sera composé par le consomation de CO2 dans le cas où l'on choisit dans quel pays le data center se trouve

In [53]:
"""
Attention ici il le dataframe entrant devra déja être passé par une des méthodes pour fonctionner.
Aussi il serai plus intéressant de faire passer le data frame par la fonction prix pour pouvoir comparer.
"""
def co2_per_region(df, df_price, df_geo, df_vm, region) :
    df_copy= df.copy()
    df_price_copy = df_price.copy()
    df_geo_copy=df_geo.copy()
    df_vm_copy=df_vm.copy()
    
    df_copy.loc[:,f'kWh']=0
    df_copy.loc[:,f'CO2 (méthode 5) if {region}']=0
    df_copy.loc[:,f'price if {region}']=0
    
    #Calcul C02 pour region spécifique (méthode 5)
    
    df_m5 = df.copy()
        
    #Calculs kWh
    df_m5 = kWh_vm_met_3(df_m5, df_vm_copy)
   
    df_m5 = kWh_sto(df_m5)
   
    df_copy.loc[:,f'kWh']=df_m5['kWh']
    
    #Ajout CO2/kWh
    for i in df_geo_copy['region'].value_counts().index :# Verifier pour les valeures nul si pris en compte
        co2= df_geo_copy['ville'][df_geo_copy['region']==region]
        co2= co2.iloc[0]#Verifier si nombre
        co2 = float(co2.replace(',', '.'))
        df_m5.loc[df_m5['region'] == i, 'CO2/kWh'] = co2
        
    #Calcul CO2
    df_m5 = co2_test(df_m5)
    
    df_copy.loc[:,f'CO2 (méthode 5) if {region}']=df_m5['CO2']
    
    
    
    #Calcul prix pour region spécifique
    
    df_prix = df_m5.copy()
    
    df_prix.loc[:,'price_final']=0
    
    for i in df_prix['usageType'][df_prix['serviceCode'].str.lower()=='virtual machines'].unique() :   
        for j in df_prix['region'][(df_prix['serviceCode'].str.lower()=='virtual machines') & (df_prix['usageType']==i)].unique():

            result=df_price_copy[region][df_price_copy['Name'] == i] #filtrer sur la machine virtuelle
            result=result.iloc[0]
            result = float(result.replace(',', '.'))
            

            df_prix.loc[(df_prix['serviceCode'].str.lower()=='virtual machines') & 
                          (df_prix['usageType'] == i) & 
                          (df_prix['region'] == j)
                          , 'price_final']= result*df_prix['usageQuantity']
    df_copy.loc[:,f'price if {region}']=df_prix['price_final']
    
    df_copy= df_copy.fillna(0)
    return df_copy
    
    

In [54]:
test = co2_per_region(prix_m4_m5, df_price, df_geo, df_vm, 'westeurope')
test = co2_per_region(test, df_price, df_geo, df_vm, 'francecentral')
test = co2_per_region(test, df_price, df_geo, df_vm, 'northeurope')
test = co2_per_region(test, df_price, df_geo, df_vm, 'westus2')

In [55]:
#test.to_csv('prix_méthode4_5_per_region.csv', index=False)